In [1]:
import pandas as pd
import numpy as np
import import_ipynb
from sklearn.preprocessing import MinMaxScaler
import random
from sampling import progressive_sample,random_sample,randu_progo,progu_rando
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense,LSTM, Input, Activation
from keras.layers import Convolution2D,MaxPooling2D, Dropout , Flatten,  Conv1D, MaxPooling1D
from keras.layers import ConvLSTM2D, BatchNormalization, MaxPooling3D,Bidirectional, GRU 
from keras import Model
from keras import metrics
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

importing Jupyter notebook from sampling.ipynb


In [2]:
cd E:\Neethu_ZU\Phase2\code_data

E:\Neethu_ZU\Phase2\code_data


In [4]:
#Read the dataset file of extracted features
data_row = pd.read_csv('data/dataset_exam_proctor_new.csv')

In [5]:
data_row.head()

,id,label,vertical_head_movement,horizontal_head_movement,vertical_eye_movement,horizontal_eye_movement,mouth_height,mouth_threshold,identity,blinking
0,1,0,9,-34,1.015873,0.614286,0.804738,7.933333,1,0
1,1,0,7,-35,1.071429,0.614286,0.804738,7.933333,1,0
2,1,0,7,-35,1.071429,0.614286,0.804738,7.933333,1,0
3,1,0,7,-31,1.000000,0.628571,0.000000,8.166667,1,0
4,1,0,7,-31,1.187500,0.632353,1.414214,7.943039,1,1


In [8]:
#Scale data

df_row_scaled = data_row.copy()
scalar = MinMaxScaler()
df_row_scaled[['vertical_head_movement', 'horizontal_head_movement',
            'vertical_eye_movement', 'horizontal_eye_movement', 'mouth_height',
            'mouth_threshold']] = scalar.fit_transform(df_row_scaled[['vertical_head_movement', 'horizontal_head_movement',
                                            'vertical_eye_movement', 'horizontal_eye_movement', 'mouth_height',
                                           'mouth_threshold']])

In [12]:
# seperate data and label of each user and save it
group = df_row_scaled.groupby(['id'])
label_df = group.first()[['label']].reset_index()
label_df.to_csv('data/user_label.csv', index= False)
data_series = [df_row_scaled.iloc[group.groups[i],2:10].values.tolist() for i in list(group.groups.keys())]
np.save("sampled_data/input_video_series.npy",data_series)

C:\Users\Neethu Venugopal\AppData\Local\Continuum\anaconda3\envs\py_img\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [3]:
data_series = np.load('sampled_data/input_video_series.npy', allow_pickle=True)
label_data = pd.read_csv('data/user_label.csv')
label_array = np.array(label_data[['label']])

In [4]:
# Make samples with equal length sequences using Random undersample oversample
data_series_eq = []
Y = []
for i,seq in enumerate(data_series):
    video_frac,l = random_sample(seq,100,label_array[i],sub_samples = None)
    if(len(video_frac)>0): 
        data_series_eq.extend(video_frac)
        Y.extend(l)

In [ ]:
seq_length = "100"
repeat = "dyn"
with open('sampled_data/random_data_v2_eq_vol/inpdata_' +seq_length+'_'+ repeat+'.npy', 'wb') as f:
    np.save(f, np.array(data_series_eq))
with open('sampled_data/random_data_v2_eq_vol/labeldata_' +seq_length+'_'+ repeat+'.npy', 'wb') as l:
    np.save(l, np.array(Y))

In [21]:
with open('sampled_data/random_data_v2/inpdata_100_dyn.npy', 'rb') as f:
        X = np.load(f)
with open('sampled_data/random_data_v2/labeldata_100_dyn.npy', 'rb') as l:
        Y = np.load(l)
SHAPE = X.shape

In [19]:
#LSTM model
def create_model():
    inp_data = Input(shape = (SHAPE[1],SHAPE[2]))
    lstm1 = LSTM(100, return_sequences = True )(inp_data)
    flat1 = (Flatten())(lstm1)
    dense1 = Dense(128, activation='relu')(flat1)
    output = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=inp_data, outputs=output)
    return model

In [22]:
# CNN model
X = X.reshape(SHAPE[0], 1, SHAPE[1],SHAPE[2])
def create_model():
    inp_data = Input(shape = (1, SHAPE[1],SHAPE[2]))
    cnn1 = (Convolution2D(32, (3, 3), activation='relu', data_format='channels_first'))(inp_data)
    pool1 = (MaxPooling2D(pool_size=(2,2)))(cnn1)
    drop1 = (Dropout(0.25))(pool1)
    cnn2 = (Convolution2D(32, (3, 3),activation='relu'))(drop1)
    pool2 = (MaxPooling2D(pool_size=(2,2)))(cnn2)
    drop2 = (Dropout(0.25))(pool2) 
    flat = (Flatten())(drop2)
    dense1 = (Dense(64, activation='relu'))(flat)
    drop3 = (Dropout(0.5))(dense1)
    output = (Dense(1, activation='sigmoid'))(drop3)
    model = Model(inputs=inp_data, outputs=output)
    return model

In [ ]:
#CNN-LSTM model
def create_model():
    inp_data = Input(shape = (SHAPE[1], SHAPE[2]))
    conv2 = (Conv1D(filters=32, kernel_size=3, activation='relu'))(inp_data)
    pool1 = (MaxPooling1D(pool_size=2))(conv2)
    lstm1 = (LSTM(100, return_sequences = True))(pool1)
    flat2 = (Flatten())(lstm1)
    drop2 = (Dropout(0.5))(flat2)
    dense1 = (Dense(128, activation='relu'))(drop2)
    output = (Dense(1, activation='sigmoid'))(dense1)
    model = Model(inputs=inp_data, outputs=output)
    return model

In [ ]:
# ConvLSTM
n_length = 4
n_steps = int(SHAPE[1]/4)
X = X.reshape((SHAPE[0], n_steps,1, n_length, SHAPE[2]))
def create_model():
    inp_data = Input(shape = (n_steps, 1, n_length, SHAPE[2]))
    convlstm1 = (ConvLSTM2D(filters=100, kernel_size=(1,2), activation='relu', data_format='channels_first', return_sequences = True))(inp_data)
    batch1 = (BatchNormalization())(convlstm1)
    pool2 = (MaxPooling3D())(batch1)
    drop1 = (Dropout(0.2))(pool2)
    flat1 = (Flatten(data_format='channels_first'))(drop1)
    dense1 = (Dense(64, activation='relu'))(flat1)
    output = (Dense(1, activation='sigmoid'))(dense1)
    model = Model(inputs=inp_data, outputs=output)
    return model

In [24]:
#Conv-BiGRU
X = X.reshape(SHAPE[0], SHAPE[1], SHAPE[2])
def create_model():
    inp_data = Input(shape = (SHAPE[1], SHAPE[2]))
    conv1 = (Conv1D(filters=32, kernel_size=(2)))(inp_data)
    activ1 = (Activation('relu'))(conv1)
    drop2 = (Dropout(0.25))(activ1)
    gru1 = (Bidirectional(GRU(32, return_sequences = True, activation='relu')))(drop2)
    flat1 = (Flatten())(gru1)
    dense1 = (Dense(64 , activation='relu'))(flat1)
    dense1 = (Dense(32 , activation='relu'))(dense1)
    dense1 = (Dense(16 , activation='relu'))(dense1)
    drop4 = (Dropout(0.4))(dense1)
    output = (Dense(1, activation='sigmoid'))(drop4)
    model = Model(inputs=inp_data, outputs=output)
    return model

In [ ]:
# evaluation using cross validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores_auc = []
cvscores_acc = []
cvscores_prec = []
cvscores_rec = []
for train, test in kfold.split(X, Y):
  # create model
	model = create_model()
	model.summary()
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam',metrics=[metrics.AUC(name = 'auc'), 
                                                                        metrics.BinaryAccuracy(name = 'accuracy'),
                                                                        metrics.Precision(name = 'precision'),
                                                                        metrics.Recall(name = 'recall')])
	# Fit the model
	print(len(X[train]),len(X[test]))
	es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
	history = model.fit(X[train], Y[train], epochs=70, batch_size=16,  validation_data=(X[test], Y[test]),callbacks=[es], verbose=1)
	# evaluate the model
	scores = model.evaluate(X[test], Y[test], verbose=1)
	print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
	print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
	print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
	print("%s: %.2f%%" % (model.metrics_names[4], scores[4]*100))
	cvscores_auc.append(scores[1] * 100)
	cvscores_acc.append(scores[2] * 100)
	cvscores_prec.append(scores[3] * 100)
	cvscores_rec.append(scores[4] * 100)
print("%s:%.2f%% (+/- %.2f%%)" % (model.metrics_names[1], np.mean(cvscores_auc), np.std(cvscores_auc)))
print("%s:%.2f%% (+/- %.2f%%)" % (model.metrics_names[2],np.mean(cvscores_acc), np.std(cvscores_acc)))
print("%s:%.2f%% (+/- %.2f%%)" % (model.metrics_names[3], np.mean(cvscores_prec), np.std(cvscores_prec)))
print("%s:%.2f%% (+/- %.2f%%)" % (model.metrics_names[4], np.mean(cvscores_rec), np.std(cvscores_rec)))

In [ ]:
# evaluation using train-test split
#split train and test data
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, shuffle = True)
model = create_model()
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=[metrics.AUC(name = 'auc'), 
                                                                        metrics.BinaryAccuracy(name = 'accuracy'),
                                                                        metrics.Precision(name = 'precision'),
                                                                        metrics.Recall(name = 'recall')])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
history = model.fit(X_train, y_train,batch_size=16,epochs=70, verbose=1, validation_data=(X_valid, y_valid),shuffle = True, callbacks=[es])
